In [5]:
%%capture

from ablator import ModelConfig, OptimizerConfig, TrainConfig, RunConfig, ParallelConfig
from ablator import ModelWrapper, ProtoTrainer, Stateless, Derived, ParallelTrainer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, accuracy_score

import os
import shutil

In [6]:
search_space = {}

model_config = ModelConfig()

optimizer_config = OptimizerConfig(
    name="adam", 
    arguments={"lr": 0.001}
)

train_config = TrainConfig(
    dataset="Fashion-mnist",
    batch_size=32,
    epochs=10,
    optimizer_config=optimizer_config,
    scheduler_config=None,
    rand_weights_init = True
)

# Random seed is used for generating same sequence of randomization every time.
run_config = ParallelConfig(
    train_config=train_config,
    model_config=model_config,
    metrics_n_batches = 800,
    experiment_dir = "/tmp/dir",
    device="cpu",
    amp=False,
    random_seed = 42,
     total_trials = 5,
    concurrent_trials = 5,
    search_space = search_space,
    optim_metrics = {"val_loss": "min"},
    gpu_mb_per_experiment = 1024,
    cpus_per_experiment = 1,
)

In [7]:
%%capture

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.ToTensor()

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)


In [8]:
# Define the model
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 64)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)
        return x

model = FashionCNN()

In [10]:
class MyModel(nn.Module):
    def __init__(self, config: ModelConfig) -> None:
        super().__init__()
       
        
        self.model = model
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, labels=None):
        out = self.model(x)
        loss = None

        if labels is not None:
            loss = self.loss(out, labels)

        out = out.argmax(dim=-1)

        return {"y_pred": out, "y_true": labels}, loss

In [11]:
def my_accuracy(y_true, y_pred):
    return accuracy_score(y_true.flatten(), y_pred.flatten())

def my_f1_score(y_true, y_pred):
    return f1_score(y_true.flatten(), y_pred.flatten(), average='weighted')

In [12]:
class MyModelWrapper(ModelWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def make_dataloader_train(self, run_config: ParallelConfig):
        return train_dataloader

    def make_dataloader_val(self, run_config: ParallelConfig):
        return test_dataloader

    def evaluation_functions(self):
        return {
            "accuracy": my_accuracy,
            "f1_score": my_f1_score
            }

In [15]:
import ray
ray.shutdown()
ray.init(address= "auto")

2023-06-15 14:24:49,154	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2023-06-15 14:24:49,166	INFO worker.py:1538 -- Connected to Ray cluster.


Python version:,3.10.6
Ray version:,2.2.0


In [16]:
import os
wrapper = MyModelWrapper(
    model_class=MyModel,
)

ablator = ProtoTrainer(
    wrapper=wrapper,
    run_config=run_config,
)
metrics = ablator.launch()

2023-06-15 14:24:50: Model directory: None
2023-06-15 14:24:50: Creating new model
2023-06-15 14:24:50: Model directory: \tmp\dir\1901_9991
2023-06-15 14:24:50: Creating new model


NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


2023-06-15 14:25:43: train_loss: 0.5935131708661715 val_loss: nan train_accuracy: 0.8591796875 train_f1_score: 0.8584602402259235 val_accuracy: nan val_f1_score: nan best_iteration: 0 best_loss: inf current_epoch: 1 current_iteration: 1875 epochs: 10 learning_rate: 0.001 total_steps: 18750
2023-06-15 14:25:48: Evaluation Step [1] train_loss: 0.5935131708661715 val_loss: 0.39709614907590723 train_accuracy: 0.8591796875 train_f1_score: 0.8584602402259235 val_accuracy: 0.8633 val_f1_score: 0.8553333168743856 best_iteration: 0 best_loss: inf current_epoch: 1 current_iteration: 1875 epochs: 10 learning_rate: 0.001 total_steps: 18750
2023-06-15 14:26:44: train_loss: 0.3267411854684353 val_loss: 0.39709614907590723 train_accuracy: 0.8726367187499999 train_f1_score: 0.8719878031317008 val_accuracy: 0.8633 val_f1_score: 0.8553333168743856 best_iteration: 1875 best_loss: 0.39709614907590723 current_epoch: 2 current_iteration: 3750 epochs: 10 learning_rate: 0.001 total_steps: 18750
2023-06-15 14: